#### **But DU MODELE:**
#### **Prédire et prévenir les blessures causées lors de matchs de tennis.**

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
chemin_excel_blessures = "../DataSets/BlessuresTennis_Causes.xlsx"
df = pd.read_excel(chemin_excel_blessures)

In [3]:
# Nettoyage des données
def fix_separators(df):
    corrections = {}
    for column in df.columns:
        corrections[column] = []
        for i, value in enumerate(df[column]):
            if pd.notnull(value) and '; ' in str(value):
                parts = str(value).split('; ')
                if len(parts) > 1:
                    df.at[i, column] = '; '.join(part.strip() for part in parts)
                    corrections[column].append(f"Ligne {i+1}: Séparateur corrigé")
            elif pd.notnull(value) and ';' in str(value):
                df.at[i, column] = '; '.join(part.strip() for part in str(value).split(';') if part.strip() != '')
                corrections[column].append(f"Ligne {i+1}: Séparateur corrigé")
    return df, corrections

df, corrections = fix_separators(df[['ID', 'CategBlessure', 'TypeBlessure', 'Blessure', 'Symptomes_Blessures', 'Traitement_Blessures', 'Prescriptions_Blessures', 'Prix_Prescriptions (en € )', 'Images_Prescriptions', 'Descriptifs_Prescriptions', 'LienAchat_Prescriptions', 'Causes_Blessure', 'Zone_Blessures', 'Description_Zone']])
print("Séparateurs corrigés :")
for column, correction in corrections.items():
    if correction:
        print(f"Colonne {column}:")
        for item in correction:
            print(f"- {item}")
        print('\n')

Séparateurs corrigés :
Colonne Blessure:
- Ligne 4: Séparateur corrigé
- Ligne 21: Séparateur corrigé
- Ligne 22: Séparateur corrigé


Colonne Symptomes_Blessures:
- Ligne 1: Séparateur corrigé
- Ligne 2: Séparateur corrigé
- Ligne 3: Séparateur corrigé
- Ligne 4: Séparateur corrigé
- Ligne 5: Séparateur corrigé
- Ligne 7: Séparateur corrigé
- Ligne 8: Séparateur corrigé
- Ligne 9: Séparateur corrigé
- Ligne 10: Séparateur corrigé
- Ligne 15: Séparateur corrigé
- Ligne 21: Séparateur corrigé
- Ligne 22: Séparateur corrigé
- Ligne 24: Séparateur corrigé
- Ligne 31: Séparateur corrigé
- Ligne 32: Séparateur corrigé
- Ligne 34: Séparateur corrigé
- Ligne 38: Séparateur corrigé


Colonne Traitement_Blessures:
- Ligne 1: Séparateur corrigé
- Ligne 6: Séparateur corrigé
- Ligne 12: Séparateur corrigé
- Ligne 18: Séparateur corrigé
- Ligne 20: Séparateur corrigé
- Ligne 24: Séparateur corrigé
- Ligne 25: Séparateur corrigé
- Ligne 26: Séparateur corrigé
- Ligne 29: Séparateur corrigé
- Ligne 

In [4]:
def extract_float(value):
    matches = re.findall(r'\d+\.\d+', str(value)) 
    if matches:
        return float(matches[0])
    else:
        return None

In [5]:
# Remplacer les virgules par des points dans les valeurs de la colonne 'Prix_Prescriptions (en € )'
df['Prix_Prescriptions (en € )'] = df['Prix_Prescriptions (en € )'].str.replace(',', '.')
df['Prix_Prescriptions (en € )'] = df['Prix_Prescriptions (en € )'].apply(extract_float)

df = df.dropna(subset=['Prix_Prescriptions (en € )'])

In [6]:
columns_to_convert = ['Prescriptions_Blessures', 'Prix_Prescriptions (en € )', 'Images_Prescriptions', 'Descriptifs_Prescriptions', 'LienAchat_Prescriptions']
df = df.dropna(subset=columns_to_convert, how='all')
df.head()

,ID,CategBlessure,TypeBlessure,Blessure,Symptomes_Blessures,Traitement_Blessures,Prescriptions_Blessures,Prix_Prescriptions (en € ),Images_Prescriptions,Descriptifs_Prescriptions,LienAchat_Prescriptions,Causes_Blessure,Zone_Blessures,Description_Zone
0,1,Blessures aux membres supérieurs,Douleur à la main et au poignet,ténosynovite de Quervain,Inflammation douloureuse des tendons près du p...,infiltrations locales de cortisone; réeduction...,Orthèse poignet-pouce tendinite NUIT; Orthèse ...,55.31,https://epitact.twic.pics/prod/2024-02/mea-que...,Soulage les douleurs liées à la tendinite De Q...,https://epitact.fr/quervimmo-orthese-immobilis...,Utilisation excessive du poignet et du pouce; ...,Zone de transition,La zone intermédiaire entre le fond de court e...
1,2,Blessures aux membres supérieurs,Douleur à la main et au poignet,ténosynovite de l’extenseur ulnaire du carpe,Inflammation douloureuse des tendons près du c...,NaN,Orthèse poignet canal carpien JOUR; Orthèse po...,23.99,https://epitact.twic.pics/prod/2024-02/mea-car...,Soulage les douleurs liées au syndrome du cana...,https://epitact.fr/carp-activ?list=Article%20T...,Surutilisation du poignet; Mauvaise posture; M...,Zone médiane gauche; Zone médiane droite,Les zones près du centre du terrain où les jou...
2,3,Blessures aux membres supérieurs,Douleur à la main et au poignet,entorse du ligament carpien,Blessure traumatique au ligament du poignet; c...,NaN,Orthèse poignet canal carpien JOUR; Orthèse po...,23.99,https://epitact.twic.pics/prod/2024-02/mea-car...,Soulage les douleurs liées au syndrome du cana...,https://epitact.fr/carp-activ?list=Article%20T...,Torsion excessive du poignet; Chute/Impact direct,Coins inférieur gauche; Coins supérieur gauche,Les coins du fond de court près du filet où le...
4,5,Blessures aux membres supérieurs,Douleur à la main et au poignet,lésions du ligament annulaire du carpe,Blessures au ligament du poignet; causant doul...,NaN,Orthèse poignet canal carpien JOUR; Orthèse po...,23.99,https://epitact.twic.pics/prod/2024-02/mea-car...,Soulage les douleurs liées au syndrome du cana...,https://epitact.fr/carp-activ?list=Article%20T...,Utilisation excessive; Mouvements répétitifs,Coins inférieur gauche; Coins supérieur gauche,Les coins du fond de court près du filet où le...
14,16,Blessures aux membres supérieurs,Douleur au dos et au ventre,Les tensions musculaires abdominales,Inconfort ou douleur dans les muscles de l'abd...,NaN,Posturostrap™ Dos,79.99,https://epitact.twic.pics/prod/2024-02/posturo...,Prévient le mal de dos lors des activités quot...,https://epitact.fr/correcteur-de-posture-postu...,Surutilisation; Mouvements brusques; Echauffem...,Coins inférieur droit; Coins supérieur droit,Les coins du fond de court où les joueurs subi...


In [8]:
def split_columns(row):
    try:
        prescriptions = row['Prescriptions_Blessures'].split('; ')
        prices = [float(price.replace(',', '.')) for price in row['Prix_Prescriptions (en € )'].split('; ')]
        images = row['Images_Prescriptions'].split('; ')
        descriptions = row['Descriptifs_Prescriptions'].split('; ')
        links = row['LienAchat_Prescriptions'].split('; ')
        return list(zip(prescriptions, prices, images, descriptions, links))
    except AttributeError:
        return [(row['Prescriptions_Blessures'], row['Prix_Prescriptions (en € )'], row['Images_Prescriptions'], row['Descriptifs_Prescriptions'], row['LienAchat_Prescriptions'])]

prescriptions_df = df.apply(split_columns, axis=1).explode().apply(lambda x: pd.Series(x, index=['Prescription', 'Prix', 'Image', 'Descriptif', 'Lien d\'achat']))

print("\nPrescriptions :")
for index, row in prescriptions_df.iterrows():
    print(f"| Prescription : {row['Prescription']} | Prix : {row['Prix']} | Image : {row['Image']} | Descriptif : {row['Descriptif']} | Lien d'achat : {row['Lien d\'achat']} |")

SyntaxError: f-string expression part cannot include a backslash (2314206287.py, line 16)

In [9]:
non_prescriptions_columns = ['Traitement_Blessures', 'Causes_Blessure', 'Zone_Blessures']
non_prescriptions_df = df[non_prescriptions_columns].reset_index().rename(columns={'index': 'Blessure'})
for col in non_prescriptions_columns:
    non_prescriptions_df[col] = non_prescriptions_df[col].str.split('; ')
non_prescriptions_df = non_prescriptions_df.explode('Traitement_Blessures').explode('Causes_Blessure').explode('Zone_Blessures')

print("\nDonnées pour le traitement, les causes et la zone des blessures :")
print(non_prescriptions_df)


Données pour le traitement, les causes et la zone des blessures :
    Blessure                Traitement_Blessures  \
0          0  infiltrations locales de cortisone   
0          0  infiltrations locales de cortisone   
0          0  infiltrations locales de cortisone   
0          0           réeduction de la pratique   
0          0           réeduction de la pratique   
..       ...                                 ...   
14        32                                 NaN   
15        33                                 NaN   
15        33                                 NaN   
15        33                                 NaN   
15        33                                 NaN   

                                 Causes_Blessure         Zone_Blessures  
0   Utilisation excessive du poignet et du pouce     Zone de transition  
0                          Mouvements répétitifs     Zone de transition  
0                   Mauvaise technique de frappe     Zone de transition  
0   Utilisat

In [10]:
categorical_columns = ['CategBlessure', 'TypeBlessure', 'Blessure', 'Symptomes_Blessures', 
                       'Traitement_Blessures', 'Prescriptions_Blessures', 'Causes_Blessure', 
                       'Zone_Blessures', 'Description_Zone']
df[categorical_columns] = df[categorical_columns].fillna('')
df[categorical_columns] = df[categorical_columns].astype(str)

encoder = OneHotEncoder()
encoded_categorical = encoder.fit_transform(df[categorical_columns])

scaled_numerical = np.array([price for price in df['Prix_Prescriptions (en € )']]).reshape(-1, 1)

X = np.concatenate((encoded_categorical.toarray(), scaled_numerical), axis=1)
y = df['Blessure']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [13]:
y_pred = model.predict(X_test)

In [14]:
# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.0
                                                       precision    recall  f1-score   support

                                        Les ampoules        0.00      0.00      0.00       0.0
Les pathologies discales dégénératives et les hernies       0.00      0.00      0.00       1.0
                          Les traumatismes de l’ongle       0.00      0.00      0.00       1.0
                                  L’entorse lombaire        0.00      0.00      0.00       0.0
                         entorse du ligament carpien        0.00      0.00      0.00       0.0
                            ténosynovite de Quervain        0.00      0.00      0.00       1.0
        ténosynovite de l’extenseur ulnaire du carpe        0.00      0.00      0.00       1.0

                                             accuracy                           0.00       4.0
                                            macro avg       0.00      0.00      0.00       4.0
                                  

c:\Users\sonia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sonia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sonia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sonia\anaconda3\Lib\site-packages\sklea

In [15]:
# Sélection des colonnes pertinentes pour la régression
X_reg = df[['CategBlessure', 'TypeBlessure', 'Blessure', 'Traitement_Blessures', 'Prescriptions_Blessures']]
y_reg = df['Prix_Prescriptions (en € )']  
X_reg_encoded = pd.get_dummies(X_reg)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg_encoded, y_reg, test_size=0.2, random_state=42)
regression_model = LinearRegression()
regression_model.fit(X_train_reg, y_train_reg)
y_pred_reg = regression_model.predict(X_test_reg)
mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)
print("Mean Squared Error (Regression):", mse_reg)
print("R^2 Score (Regression):", r2_reg)

Mean Squared Error (Regression): 102.85435703526028
R^2 Score (Regression): 0.8455893462584508


In [16]:
observation_index = 0  x
observation_features = X_test[observation_index]  x
observation_price = y_test[observation_index] 

predicted_price = model.predict([observation_features])
print("Observation - Caractéristiques:", observation_features)
print("Prix réel de prescription:", observation_price)
print("Prédiction du prix de prescription:", predicted_price)

SyntaxError: invalid syntax (993302240.py, line 1)